# Regularization

Operator inference is an inverse problem: it seeks to learn a system (or a low-dimensional representation thereof) that produced a set of observed data.
Like most inverse problems, operator inference often benefits from regularization to balance data fitting with desirable conditioning.
In some cases, the quality of an operator inference model can be quite sensitive to the choice of the regularization.
This tutorial shows how to specify regularization and demonstrated principled methods for optimal regularization selection.

**Key references:** {cite}`mcquarrie2021combustion,qian2022pdes,sawant2023pireg`.

## Problem Statement

We start with a quadratic problem.

:::{admonition} Governing Equations
:class: info

Let $\Omega = [0,L]\subset \mathbb{R}$ be the spatial domain indicated by the variable $x$, and let $[0,T]\subset\mathbb{R}$ be the time domain with variable $t$. We consider the one-dimensional Euler equations for the incompressible flow of an ideal gas with periodic boundary conditions.
The state is given by

$$
\begin{aligned}
    \vec{q}_c(x, t) = \left[\begin{array}{c}
        \rho \\ \rho v \\ \rho e
    \end{array}\right],
\end{aligned}
$$

where $\rho = \rho(x,t)$ is the density, $v = v(x,t)$ is the fluid velocity, and $e = e(x, t)$ is the internal energy.
The state evolves according in time according to the system

$$
\begin{aligned}
    \frac{\partial\vec{q}_c}{\partial t}
    = \frac{\partial}{\partial t} \left[\begin{array}{c}
        \rho \\ \rho v \\ \rho e
    \end{array}\right]
    &= -\frac{\partial}{\partial x}\left[\begin{array}{c}
        \rho v \\ \rho v^2 + p \\ (\rho e + p) v
    \end{array}\right]
    & x &\in\Omega,\quad t\in[0,T],
    \\
    \vec{q}_c(0,t) &= \vec{q}_c(L,t)
    & t &\in[0,T],
    \\
    \vec{q}_c(x,0) &= \vec{q}_0
    & x &\in \Omega,
\end{aligned}
$$

where $p = p(x,t)$ is the pressure.
Note that the dynamics are nonpolynomially nonlinear with respect to $\vec{q}_c$.
The state variables are related via the ideal gas law

$$
\begin{aligned}
    \rho e = \frac{p}{\gamma - 1} + \frac{1}{2}\rho v^{2},
\end{aligned}
$$

where $\gamma = 1.4$ is the heat capacity ratio.
:::

:::{admonition} Objective
:class: info

Construct a reduced-order model (ROM) which can be solved rapidly to produce approximate solutions $\vec{q}(x, t)$ to the partial differential equation given above for various choices of the initial condition $\vec{q}_0$.
In addition, we will only observe data over a limited time interval $t \in [0, T']$ with $T' < T$, then use the ROM to predict the solution for the entire time domain $[0, T]$.
Hence, the ROM will be **predictive in time** and **predictive in the initial data**.
:::

In [1]:
import itertools
import numpy as np
import scipy.integrate
import scipy.interpolate
import matplotlib.colors
import matplotlib.pyplot as plt

import opinf

opinf.utils.mpl_config()

## Full-order Model Definition

To generate data, we discretize the spatial derivative using a first-order forward difference method.

:::{dropdown} Discretization details

We take an equidistant grid $\{x_i\}_{i=0}^{n_x} \subset \Omega$,

$$
\begin{aligned}
    0 &= x_0 < x_1 < \cdots < x_n < x_{n_x} = L
    &
    &\text{and}
    &
    \delta x &= \frac{L}{n} = x_{i+1} - x_{i},\quad i=0,\ldots,n_x-1.
\end{aligned}
$$

The periodic boundary conditions prescribe $q(x_0,t) = q(x_{n_x},t)$.
Our goal is to compute $\vec{q}_\text{c}(x,t)$ at the spatial points $x_{0},x_{1},\ldots,x_{n_x-1}$ for each state variable in $\vec{q}_c$, so we consider the discretized state vector

$$
\begin{aligned}
    \q(t) = \left[\begin{array}{c}
        \rho(x_{0}, t) \\ \vdots \\ \rho(x_{n-1}, t)
        \\
        (\rho v)(x_{0}, t) \\ \vdots \\ (\rho v)(x_{n-1}, t)
        \\
        (\rho e)(x_{0}, t) \\ \vdots \\ (\rho e)(x_{n-1}, t)
    \end{array}\right]
    \in\RR^{3n_x}.
\end{aligned}
$$

Hence, the dimension of the full-order model is $n = 3n_x$.
The spatial derivative is approximated with a first-order backward finite difference approximation,

$$
\begin{aligned}
    \frac{\partial}{\partial x}y(x,t)
    \approx \frac{y(x, t) - y(x - \delta x, t)}{\delta x},
\end{aligned}
$$

for $y = \rho v,$ $\rho v^2 + p,$ and $(\rho e + p) v.$

Note that we do not explicitly construct matrices defining the evolution of $\q(t).$
The system is integrated in time using an adaptive Runge--Kutta method via {func}`scipy.integrate.solve_ivp` with ``method="RK45"``.
:::

### Training Data Generation

Let $L = 2$ with $n_x = 1001$ and $T = 0.15.$
We begin by solving the FOM described above, recording the solution at $K = 500$ uniformly spaced time instances, including the initial condition.
We will assume that we can only observe the first $k = 200$ snapshots (up to $T' = 0.06$).

In [ ]:
gamma = 1.4

# Construct the spatial domain.
L = 2
nx = 200  # 1000
x = np.linspace(0, L, nx + 1)[:-1]
dx = x[1] - x[0]

# Construct the temporal domain.
T = 0.15
K = 501
t_all = np.linspace(0, T, K)
dt = t_all[1] - t_all[0]


def _ddx(var):
    """First-order backward differences with periodic boundary conditions."""
    return (var - np.roll(var, 1, axis=0)) / dx


def fom_derivative(tt, state):
    """Right-hand side of the full-order model equations."""
    rho, rho_u, rho_e = np.split(state, 3)
    u = rho_u / rho
    p = (gamma - 1) * (rho_e - 0.5 * rho * u**2)

    return -np.concatenate(
        [
            _ddx(rho_u),
            _ddx(rho * u**2 + p),
            _ddx((rho_e + p) * u),
        ]
    )


def full_order_solve(q_init, time_domain):
    """Solve the full-order model with SciPy."""
    return scipy.integrate.solve_ivp(
        fun=fom_derivative,
        t_span=[time_domain[0], time_domain[-1]],
        y0=q_init,
        method="RK45",
        t_eval=time_domain,
        rtol=1e-5,
        atol=1e-8,
    ).y


# Construct initial conditions.
def initial_conditions(init_params):
    r"""Generate initial conditions by evaluating periodic cubic splines
    for density and velocity.

    Parameters
    ----------
    init_params : (6,) ndarray
        Degrees of freedom for the initial conditions, three interpolation
        values for the density and three for the velocity (in that order).

    Returns
    -------
    init : (3nx,) ndarray
        Initial conditions (in the conservative variables [rho, rho v, rho e]).
    """
    # Extrac initial condition parameters and enforce periodicity.
    rho0s, v0s = init_params[:3], init_params[3:6]
    v0s = np.concatenate((v0s, [v0s[0]]))
    rho0s = np.concatenate((rho0s, [rho0s[0]]))

    # Construct initial conditions for each variable.
    nodes = np.array([0, L / 3, 2 * L / 3, L]) + x[0]
    rho = scipy.interpolate.CubicSpline(nodes, rho0s, bc_type="periodic")(x)
    v = scipy.interpolate.CubicSpline(nodes, v0s, bc_type="periodic")(x)
    p = 1e5 * np.ones_like(x)
    rho_e = p / (gamma - 1) + 0.5 * rho * v**2

    return np.concatenate((rho, rho * v, rho_e))


# Solve the full-order model for a variety fo initial conditions.
q0s = [
    initial_conditions(icparams)
    for icparams in itertools.product(
        [20, 24],
        [22],
        [20, 24],
        [95, 105],
        [100],
        [95, 105],
    )
]
with opinf.utils.TimedBlock(f"{len(q0s)} full-order solves"):
    Q_alls = [full_order_solve(q0, t_all) for q0 in q0s]

# Solve the full-order model for a single initial condition.
q0 = initial_conditions([22, 20, 24, 95, 105, 100])
with opinf.utils.TimedBlock("Full-order solve"):
    Q_all = full_order_solve(q0, t_all)

# Retain only the first k snapshots for training the ROM.
k = 200
t_train = t_all[:k]
Q_train = Q_all[:, :k]
Q_trains = [Q[:, :k] for Q in Q_alls]

print(f"\nSpatial domain:\t\t{x.shape=}")
print(f"Spatial step size:\t{dx=:.10f}")
print(f"\nFull time domain:\t{t_all.shape=}")
print(f"Training time domain:\t{t_train.shape=}")
print(f"Temporal step size:\t{dt=:f}")
print(f"\nAll FOM solutions:\t{Q_all.shape=}")
print(f"Training snapshots:\t{Q_train.shape=}")

In [3]:
class EulerLifter(opinf.lift.LifterTemplate):
    """Lifting map for the Euler equations transforming conservative
    variables to specific volume variables.
    """

    num_original_variables = 3
    num_lifted_variables = 3

    def __init__(self, gamma=1.4):
        """Store the heat capacity ratio, gamma."""
        self.gamma = gamma

    def lift(self, state):
        """Map the conservative variables to the learning variables,
        [rho, rho*u, rho*e] -> [u, p, 1/rho].
        """
        rho, rho_u, rho_e = np.split(state, self.num_original_variables)

        u = rho_u / rho
        p = (self.gamma - 1) * (rho_e - 0.5 * rho * u**2)
        zeta = 1 / rho

        return np.concatenate((u, p, zeta))

    def unlift(self, upzeta):
        """Map the learning variables to the conservative variables,
        [u, p, 1/rho] -> [rho, rho*u, rho*e].
        """
        u, p, zeta = np.split(upzeta, self.num_lifted_variables)

        rho = 1 / zeta
        rho_u = rho * u
        rho_e = p / (self.gamma - 1) + 0.5 * rho * u**2

        return np.concatenate((rho, rho_u, rho_e))


lifter = EulerLifter()

The following code visualizes the training data and the full FOM solution set by plotting a few snapshots over the spatial domain and the time evolution of the snapshots at a few spatial locations.

In [4]:
def plot_traces(t, states, nlocs=20, lift=False):
    """Plot traces in time at ``nlocs`` locations."""
    if lift:
        states = lifter.lift(states)

    # Choose spatial locations at which to plot each state.
    xlocs = np.linspace(0, states.shape[0] // 3, nlocs + 1, dtype=int)[:-1]
    xlocs += xlocs[1] // 2
    colors = plt.cm.twilight(np.linspace(0, 1, nlocs + 1)[:-1])
    u, p, zeta = np.split(states, 3)

    # Plot the states in time.
    fig, axes = plt.subplots(3, 1, sharex=True, figsize=(12, 6))
    for j, c in zip(xlocs, colors):
        axes[0].plot(t, u[j], color=c, lw=1)
        axes[1].plot(t, p[j], color=c, lw=1)
        axes[2].plot(t, zeta[j], color=c, lw=1)

    # Format axes.
    axes[2].set_xlabel(r"$t\in[t_{0},t_{f}]$")
    axes[2].set_xlim(t[0], t[-1])
    axes[0].set_ylabel(r"velocity $v(x)$")
    axes[1].set_ylabel(r"pressure $p(x)$")
    axes[2].set_ylabel(r"1/density $\rho(x)$")
    fig.align_ylabels(axes)

    # Colorbar.
    lsc = plt.cm.twilight(np.linspace(0, 1, 400))
    scale = matplotlib.colors.Normalize(vmin=0, vmax=1)
    lscmap = matplotlib.colors.LinearSegmentedColormap.from_list(
        "euler", lsc, N=nlocs
    )
    mappable = plt.cm.ScalarMappable(norm=scale, cmap=lscmap)
    cbar = fig.colorbar(mappable, ax=axes, pad=0.015)
    cbar.set_ticks(x[xlocs] / (x[-1] - x[0]))
    cbar.set_ticklabels([f"{xx:.2f}" for xx in x[xlocs]])
    cbar.set_label(r"spatial coordinate $x$")

    return fig, axes


def plot_regimes(states, nlocs=20, lift=False):
    """Plot traces in time and label training/prediction regimes."""
    fig, axes = plot_traces(t_all, states, nlocs, lift)

    for ax in axes.flat:
        ax.axvline(t_train[-1], color="black", linewidth=1)
    ymax = axes[0].get_ylim()[1]
    axes[0].text(t_train[-1] / 2, ymax, "training regime", ha="center")
    xpred = (t_all[-1] - t_train[-1]) / 2 + t_train[-1]
    axes[0].text(xpred, ymax, "prediction regime", ha="center")

    return fig, axes

In [ ]:
plot_traces(t_train, Q_train, nlocs=20, lift=True)
plt.show()

In [ ]:
plot_regimes(Q_all, lift=True)
plt.show()

### ROM Construction

We will use a single (monolithic) POD basis for all three lifted state variables jointly.
That means we need to scale our data appropriately first.

We select characteristic scales $\bar{v} = 100~[\text{m}/\text{s}]$ for the velocity and $\bar{p} = 100000~[\text{Pa}]$ for the pressure.
As can be seen from the ideal gas law, the characteristic scale for density is

$$
    \bar{\rho}
    = \frac{\bar{p}}{\bar{v}^2}
    = 10.
$$

In [7]:
euler_scaler = opinf.pre.TransformerMulti(
    transformers=[
        opinf.pre.ScaleTransformer(1e-2, name="velocity"),
        opinf.pre.ScaleTransformer(1e-5, name="pressure"),
        opinf.pre.ScaleTransformer(1e1, name="specific volume"),
    ]
)

In [8]:
euler_shifter = opinf.pre.ShiftScaleTransformer(centering=True)
euler_transformer = opinf.pre.TransformerPipeline(
    [euler_shifter, euler_scaler]
)

We're now ready to construct a reduced-order model.
Note that the model form is purely quadratic.

**Can either shift and use 8 basis vectors and full structure, or no shift and use 9 basis vectors and purely quadratic structure**

In [ ]:
rom = opinf.ROM(
    lifter=lifter,
    # transformer=euler_transformer,
    transformer=euler_scaler,
    # basis=opinf.basis.PODBasis(num_vectors=8),
    basis=opinf.basis.PODBasis(num_vectors=9),
    ddt_estimator=opinf.ddt.UniformFiniteDifferencer(t_train, "ord6"),
    model=opinf.models.ContinuousModel(
        # operators="cAH",
        operators="H",
        solver=opinf.lstsq.L2Solver(regularizer=1e-8),
    ),
).fit(Q_trains)

print(rom)

Some sanity checks: basis energy and initial condition projection error.

In [ ]:
rom.basis.plot_energy(right=20)
plt.show()

In [ ]:
fig, axes = plt.subplots(1, 3)
for i, ax in enumerate(axes.flat):
    ax.plot(x, np.split(q0, 3)[i])
    ax.plot(x, np.split(rom.project(q0), 3)[i])
plt.show()

The quality of the ROM is sensitive to the regularization hyperparameter, which we initially set to `1e-8`.


**Actually, the quality of the ROM is super sensitive to the number of basis vectors, even with multiple data trajectories. This is not a good example for regularization, but it is a good example for lifting.**

In [ ]:
rom.fit_regselect(
    Q_trains,
    candidates=np.logspace(-16, 3, 20),
    fit_basis=True,
    fit_transformer=True,
    # gridsearch_only=True,
    train_time_domains=t_train,
    test_time_length=0.09,
    stability_margin=5,
    verbose=True,
)
print(
    f"\nROM error, "
    f"r = {rom.basis.reduced_state_dimension}, "
    f"reg = {rom.model.solver.regularizer:.4e}:"
)
for ell, Q in enumerate(Q_alls):
    Q_rom = rom.predict(Q[:, 0], t_all)
    if Q_rom.shape[1] != t_all.size:
        print(f"  trial {ell: >2d}: time integration failed ")
    else:
        print(
            f"  trial {ell: >2d} error:",
            f"{opinf.post.Lp_error(Q, Q_rom, t_all)[1]:.6%}",
        )
        # fig, axes = plot_regimes(Q_rom, lift=True)
        # plt.show()

In [ ]:
# plot_regimes(Q_rom - Q_alls[-1], lift=False)
plot_regimes(Q_rom, lift=True)
plot_regimes(Q_alls[-1], lift=True)
plt.show()